## Orbital parameters of merging systems

The effect of a merger (on galaxy spin) should depend on orbital parameter of the merger. 

+ Retro / Pro
+ co-/counter rotating

Trajectories of merging systems can be extracted from the merger tree (If you believe the Tree!). 

In [1]:
cd /home/hoseung/Work/data/29172

/media/hoseung/btrfs/29172


In [2]:
import pickle
#%matplotlib notebook
import matplotlib 
matplotlib.use("Qt4Agg")
import matplotlib.pyplot as plt
import numpy as np
# Need a full tree. not only main progenitor tree. 
tt = pickle.load(open("./GalaxyMaker/Trees/extended_tree.pickle", "rb"))
from tree import ctutils as ctu
from galaxy.galaxy import Galaxy
import tree.halomodule as hmo
#alltree = extract_a_tree(tt, idx=)

In [3]:
# Final root galaxies
t_final = tt.data[tt.data["nout"] == 187]

# Choose a random galaxy tree
id_root = t_final["id"][40]
print("Chosen id_root = ", id_root)
atree = ctu.extract_a_tree(tt.data, id_root)
trunk = ctu.extract_main_tree(atree, id_root)


allbranches=[]
# Iterate over a full tree to seprate out each branch.
while len(atree) > 10:
    abranch = ctu.extract_main_tree(atree, atree["id"][0], no_subset=True)
    # Need no_subset=True.
    # A subset of the whole tree is taken by default to reduce the search time. 
    # But the subset is extracted by checking the "tree_root_id".
    # Because the root of "each" branch is not the root of the tree, subset shouldn't be taken. 
    
    allbranches.append(abranch)    
    # I do not extract all branches because some of the branches are too short.
    # I'll ignore them. 
    # Mask extracted tree, and leave the short branches there.
    for gal in atree:
        if gal["id"] in abranch["id"]:
            #print(gal["id"])
            gal["id"] = 1 
    
    # keep the rest(short branches)
    atree = atree[atree["id"] != 1]

Chosen id_root =  43049


In the following loop, 
branch(2) = branch[i_nout_ok] is a copy of the original branch(1), which is an i-th element of the allbranches. 
Thus, modifying a copy of the branch does nothing to allbranches. 
To update allbranches, put the new branch(2) into the allbranches, and then modify the branch(2).

In [4]:
#trunk = allbranches[0]
min_nout_trunk = min(trunk["nout"])
for i, branch in enumerate(allbranches):
    i_nout_ok = np.where(branch["nout"] >= min_nout_trunk)[0]
    branch = branch[i_nout_ok] # this is a new branch.
    allbranches[i] = branch
    for gal in branch:
        i = np.where(trunk["nout"] == gal["nout"])[0]
        gal["x"] -= trunk["x"][i]
        gal["y"] -= trunk["y"][i]
        gal["z"] -= trunk["z"][i]
        gal["vx"] -= trunk["vx"][i]
        gal["vy"] -= trunk["vy"][i]
        gal["vz"] -= trunk["vz"][i]

Branches can start earlier than the trunk. For those points, X_brn - X_trn is impossible. 
Let me ignore then for the moment. 

In [5]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for branch in allbranches[1:]:
    ax.scatter(branch["x"], branch["y"], branch["z"], s=branch["m"]/5e8, alpha=0.5)

ax.scatter(0,0,0,s=10000, c='r', alpha=0.3)
plt.show()

In [6]:
branch = allbranches[1]

fig, ax = plt.subplots()

ax.quiver(branch["x"], branch["y"], branch["vx"], branch['vy'])
ax.scatter(branch["x"], branch["y"])

plt.show()

branches are separated.
So far, so good.

### Now, calculate rotation vector of each galaxy

1. load a single galaxy (rd_gal)

2. cross product

3. complete 1,2 steps for the trunk and a branch

In [7]:
import load
import os
from galaxy import rd_gal
from utils import match as mtc

In [8]:
igal = 0
idgal = trunk["Orig_halo_id"][igal]
nout = trunk["nout"][igal]
header, stars = rd_gal.rd_gal(187, idgal)

Note that header is a numpy array of length 1, but not a recarray.
Stars is a long recarray.

In [9]:
print(header.dtype)
print(stars.dtype)

stars["pos"] -= header["xg"] # It's as simple as this!
stars["vel"] -= header["vg"]
stars["pos"] *=1e3 # in kpc

[('my_number', '<i4'), ('level', '<i4'), ('mgal', '<f8'), ('xg', '<f8', (3,)), ('vg', '<f8', (3,)), ('lg', '<f8', (3,)), ('npart', '<i4')]
(numpy.record, [('pos', '<f8', (3,)), ('vel', '<f8', (3,)), ('id', '<i4'), ('m', '<f8'), ('time', '<f8'), ('metal', '<f8')])


In [10]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

i_sub = range(1,len(stars),100) # 1 in every 100.

X,Y,Z = stars["pos"][i_sub,0],stars["pos"][i_sub,1],stars["pos"][i_sub,2]
U,V,W = stars["vel"][i_sub,0],stars["vel"][i_sub,1],stars["vel"][i_sub,2]
# Rotation axis of the galaxy
vec_rot = np.cross(stars["pos"],stars["vel"])
Nx, Ny, Nz = vec_rot[i_sub,0], vec_rot[i_sub,1], vec_rot[i_sub,2]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# velocity
scale_v = 3e-2
ax.quiver(X,Y,Z,
          U*scale_v,
          V*scale_v,
          W*scale_v,
          color="blue", pivot="tail")

# Normal vectors
scale_n = 1 # No need to rescale, because it is normalized anyways.
ax.quiver(X,Y,Z,
          Nx*scale_n,
          Ny*scale_n,
          Nz*scale_n,
          color="red",
          pivot="tail", 
          normalize=True)


Ln = vec_rot.sum(axis=0)
Nn = Ln / np.linalg.norm(Ln)
ax.quiver(0,0,0,Nn[0], Nn[1], Nn[2],
          color="green", pivot="tail",
          length = 30)

plt.show()

In [12]:
def get_normal_vec(stars):
    vec_rot = np.cross(stars["pos"],stars["vel"])
    Ln = vec_rot.sum(axis=0) 
    return Ln / np.linalg.norm(Ln)

def plot_n_arrows(ax, branch, trunk=None, color="blue", cmap=None):
    """
        Plots normal vectors of the branch over time. 
        If trunk is not None, galaxy positions relative to the host(trunk) are used. 
    """
    
    import matplotlib.colors as colors
    import matplotlib.cm as cmx
    from collections import Iterable

    if isinstance(color, Iterable) and not isinstance(color, (str)):
        color_arr = True
        cNorm  = colors.Normalize(vmin=np.min(color), vmax=np.max(color))
        scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)
    else:
        color_arr = False
        colorVal = color
    
    for i, brick in enumerate(branch):
        # load gal file
        idgal = brick["Orig_halo_id"]
        nout = brick["nout"]
        try:
            header, stars = rd_gal.rd_gal(nout, idgal)
        except:
            print("Failed to load galaxy file, skipping...")
            return False
        info = load.info.Info(nout=nout)
        
        
        # calculate Vec_rot
        # Should I take all stellar particles?
        nv = get_normal_vec(stars)

        abs_pos_gal = (header["xg"] + 100)#0.5*info.pboxsize)

        if trunk is not None:
            trunk_this = trunk[trunk["nout"] == nout]        
            abs_pos_trunk = (trunk_this["x"][0],
                             trunk_this["y"][0],
                             trunk_this["z"][0])
            rel_pos_gal = abs_pos_gal - abs_pos_trunk # pkpc
        else:
            rel_pos_gal = abs_pos_gal

        if color_arr:
            colorVal = scalarMap.to_rgba(color[i])
        ax.quiver(rel_pos_gal[0],
                  rel_pos_gal[1],
                  rel_pos_gal[2],
                  nv[0], nv[1], nv[2],
                  color=colorVal, pivot="tail")


In [13]:
def get_pericenter(trunk, branch, return_dist=False):
    """
    returns the index at the pericenter,
    or the distance if return_dist = True.
    """
    dist = np.sqrt(np.square(trunk["x"]-branch["x"]) + 
                   np.square(trunk["y"]-branch["y"]) + 
                   np.square(trunk["z"]-branch["z"]))    
    from scipy.signal import argrelextrema
    search_window = 2 # number of neighboring points to test against.
    if return_dist:
        i_min = argrelextrema(dist, np.less, order=search_window)
        return dist[i_min]
    else:
        return argrelextrema(dist, np.less, order=search_window)

    
def trunk_counter_part(trunk, branch):
    """
    returns a part of trunk of overlaping nouts.
    """
    i_counterpart = mtc.match_list_ind(trunk["nout"], branch["nout"])
    return trunk[i_counterpart]
    
  

In [14]:
nv_trunk=[]
for brick in trunk:
    idgal_b = brick["Orig_halo_id"]
    nout = brick["nout"]
    info = load.info.Info(nout=nout)
    header_b, stars_b = rd_gal.rd_gal(nout, idgal_b)

    nv_b = get_normal_vec(stars_b)
    nv_trunk.append(nv_b)

# Make an augmented recarray
dt = trunk.dtype.descr
dt.append(("nv", "f", (3,)))
new_arr = np.empty(len(trunk), dtype=dt)

import numpy.lib.recfunctions as rf
rf.recursive_fill_fields(trunk,new_arr) # copy original data
new_arr["nv"] = np.array(nv_trunk) # assign new data
trunk = new_arr

### Individual, animated version

In [19]:
import matplotlib.colors as colors
import matplotlib.cm as cmx

fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)

#colors=["black", "gray", "silver", "maroon", "red", "purple", "fuchisica", "lime", "olive"]

#plot_n_arrows(ax1, trunk_part, trunk=None, color="black")

ignore_position=True # put two n vectors at the origin. 
nout_min = 30

cmap = None
cNorm  = colors.Normalize(vmin=0, vmax=90) # 0 ~ 90 degree -> -90 ~ +90 or 0 ~ 180.
scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)

for i,branch in enumerate(allbranches[1:]):
    #plot_n_arrows(ax1, branch[branch["nout"] > nout_min], trunk=None, color=colors[i])
    nv_b = []
    nv_t=[]
    rel_ang =[]

    trunk_part = trunk_counter_part(trunk, branch)
    s_idx_branch = str(branch["id"][0])

    if not os.path.isdir(s_idx_branch):
        os.mkdir(s_idx_branch)
    # Earlier time earlier
    for brick, brick_t in zip(branch[::-1], trunk_part[::-1]):
        # load gal file
        idgal_b = brick["Orig_halo_id"]
        nout = brick["nout"]
        if nout < nout_min:
            continue
        try:
            header_b, stars_b = rd_gal.rd_gal(nout, idgal_b)
        except:
            print("Failed to load galaxy file, skipping...")
            rel_ang.append(np.nan)
            continue
        info = load.info.Info(nout=nout)

        
        # rotation axis
        nvb = get_normal_vec(stars_b)
        nv_b.append(nvb)

        nvt = brick_t["nv"]
        nv_t.append(nvt)
        relang = 180 / np.pi * np.arccos(np.dot(nvb, nvt))
        rel_ang.append(relang) # Normal vectors are already normalized.

        # Position
        abs_pos_gal = (header["xg"] + 100)#0.5*info.pboxsize)
        abs_pos_trunk = (brick_t["x"],
                         brick_t["y"],
                         brick_t["z"])
        rel_pos_gal = abs_pos_gal - abs_pos_trunk
        
        # Orbital angle.
        rel_vel_gal = header["vg"] - (brick_t["vx"],brick_t["vy"],brick_t["vz"]) # km/s
        j_orbital = np.cross(rel_pos_gal, rel_vel_gal)

        # sat normal vector
        if ignore_position:
            ax1.quiver(0,0,0, nvb[0], nvb[1], nvb[2],
                color=scalarMap.to_rgba(relang), pivot="tail")
        else:
            ax1.quiver(rel_pos_gal[0],
                       rel_pos_gal[1],
                       rel_pos_gal[2],
                       color=scalarMap.to_rgba(relang), pivot="tail")         
        
        # host normal vector
        ax1.quiver(0,0,0, brick_t["nv"][0],brick_t["nv"][1],brick_t["nv"][2],
                  color="black", pivot="tail")
        
        ax2.plot(rel_ang, color = "b")
        ax2.set_xlim([0,len(branch) + 5])
        ax2.set_ylim([0,180])
        ax1.auto_scale_xyz([-1.5,1.5],[-1.5,1.5],[-1.5,1.5])
        ax1.set_xlabel("X")
        ax1.set_ylabel("Y")
        ax1.set_zlabel("Z")
        

        plt.savefig(s_idx_branch + "/" + s_idx_branch + "_" + str(nout))
        ax1.clear()
    ax2.clear()

KeyboardInterrupt: 

From the figure, the relative angles are measured right, up to the magnitude. Add sign. 

BTW, the second branch tree seems imperfect. It's motion w.r.t the main galaxy makes no sense during ~ 20 snapshots.

The plot of arrays of normal vectors are informative, but an animation with a rough galaxy shape added would be better.

In [ ]:
def plot_n_arrow_single(ax, brick, trunk=None, color="blue", cmap=None):
    """
        Plots normal vectors of the branch over time. 
        If trunk is not None, galaxy positions relative to the host(trunk) are used. 
    """
    
    import matplotlib.colors as colors
    import matplotlib.cm as cmx
    from collections import Iterable

    if isinstance(color, Iterable) and not isinstance(color, (str)):
        color_arr = True
        cNorm  = colors.Normalize(vmin=np.min(color), vmax=np.max(color))
        scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)
    else:
        color_arr = False
        colorVal = color
    
    # load gal file
    idgal = brick["Orig_halo_id"]
    nout = brick["nout"]
    info = load.info.Info(nout=nout)
    header, stars = rd_gal.rd_gal(nout, idgal)

    # calculate Vec_rot
    # Should I take all stellar particles?
    nv = get_normal_vec(stars)

    abs_pos_gal = (header["xg"] + 100)#0.5*info.pboxsize)

    if trunk is not None:
        abs_pos_trunk = (trunk["x"],
                         trunk["y"],
                         trunk["z"])
        rel_pos_gal = abs_pos_gal - abs_pos_trunk
    else:
        rel_pos_gal = abs_pos_gal

    if color_arr:
        colorVal = scalarMap.to_rgba(color[i])
        
    ax.quiver(rel_pos_gal[0],
              rel_pos_gal[1],
              rel_pos_gal[2],
              nv[0], nv[1], nv[2],
              color=colorVal, pivot="tail")
    
    return nv


In [35]:
# Orbital parameter
import matplotlib.colors as colors
import matplotlib.cm as cmx

fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(221, projection='3d')
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223, projection='3d')

#colors=["black", "gray", "silver", "maroon", "red", "purple", "fuchisica", "lime", "olive"]

#plot_n_arrows(ax1, trunk_part, trunk=None, color="black")

ignore_position=True # put two n vectors at the origin. 
nout_min = 30

cmap = None
cNorm  = colors.Normalize(vmin=0, vmax=90) # 0 ~ 90 degree -> -90 ~ +90 or 0 ~ 180.
scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)

for i,branch in enumerate(allbranches[2:]):
    #plot_n_arrows(ax1, branch[branch["nout"] > nout_min], trunk=None, color=colors[i])
    nv_b = []
    nv_t=[]
    rel_ang =[]
    ang_orb=[]
    pos_arr=[]

    trunk_part = trunk_counter_part(trunk, branch)
    s_idx_branch = str(branch["id"][0])

    if not os.path.isdir(s_idx_branch):
        os.mkdir(s_idx_branch)
    # Earlier time earlier
    for brick, brick_t in zip(branch[::-1], trunk_part[::-1]):
        # load gal file
        idgal_b = brick["Orig_halo_id"]
        nout = brick["nout"]
        if nout < nout_min:
            continue
        try:
            header_b, stars_b = rd_gal.rd_gal(nout, idgal_b)
        except:
            print("Failed to load galaxy file, skipping...")
            rel_ang.append(np.nan)
            continue
        info = load.info.Info(nout=nout)

        
        # rotation axis
        nvb = get_normal_vec(stars_b)
        nv_b.append(nvb)

        nvt = brick_t["nv"]
        nv_t.append(nvt)
        relang = 180 / np.pi * np.arccos(np.dot(nvb, nvt))
        rel_ang.append(relang) # Normal vectors are already normalized.

        # Position
        abs_pos_gal = (header["xg"] + 100)#0.5*info.pboxsize)
        abs_pos_trunk = (brick_t["x"],
                         brick_t["y"],
                         brick_t["z"])
        rel_pos_gal = abs_pos_gal - abs_pos_trunk
        
        # Orbital angle.
        rel_vel_gal = header["vg"] - (brick_t["vx"],brick_t["vy"],brick_t["vz"]) # km/s
        v_orb = np.cross(rel_pos_gal, rel_vel_gal) 
        v_orb /= np.linalg.norm(v_orb)
        ang_orbital = 180 / np.pi * np.arccos(np.dot(v_orb, nvt))
        ang_orb.append(ang_orbital)

        # sat normal vector
        if ignore_position:
            ax1.quiver(0,0,0, nvb[0], nvb[1], nvb[2],
                color=scalarMap.to_rgba(relang), pivot="tail")
        else:
            ax1.quiver(rel_pos_gal[0],
                       rel_pos_gal[1],
                       rel_pos_gal[2],
                       nvb[0], nvb[1], nvb[2],
                       color=scalarMap.to_rgba(relang), pivot="tail")         
        
        # normal vectors of galaxies
        ax1.quiver(0,0,0, brick_t["nv"][0],brick_t["nv"][1],brick_t["nv"][2],
                  color="black", pivot="tail")
        
        ax1.auto_scale_xyz([-1.5,1.5],[-1.5,1.5],[-1.5,1.5])
        ax1.set_xlabel("X")
        ax1.set_ylabel("Y")
        ax1.set_zlabel("Z")
        
        # angles b/w two gals, and orbit.
        ax2.plot(rel_ang, color = "b", label="rel_ang")
        ax2.plot(ang_orb, color = "r", label="ang_orb")
        ax2.set_xlim([0,len(branch) + 5])
        ax2.set_ylim([0,180])
        ax2.legend()
        
        # Trajectories
        pos_arr.append(rel_pos_gal)
        ax3.scatter([pp[0] for pp in pos_arr],
                    [pp[1] for pp in pos_arr],
                    [pp[2] for pp in pos_arr])
        
        ax3.quiver(0,0,0, brick_t["nv"][0],brick_t["nv"][1],brick_t["nv"][2],
                    color="black", pivot="tail")
        
        nrvg = rel_vel_gal / np.linalg.norm(rel_vel_gal)
        ax3.quiver(rel_pos_gal[0],
                   rel_pos_gal[1],
                   rel_pos_gal[2],
                   nrvg[0], nrvg[1], nrvg[2],
                   color=scalarMap.to_rgba(relang), pivot="tail")
        
        plt.savefig(s_idx_branch + "/" + s_idx_branch + "_" + str(nout) + ".png")
        ax1.clear()
        #print(ang_orb)
        ax2.clear()
        ax3.clear()


Failed to load galaxy file, skipping...
Failed to load galaxy file, skipping...
Failed to load galaxy file, skipping...
Failed to load galaxy file, skipping...
Failed to load galaxy file, skipping...
Failed to load galaxy file, skipping...
Failed to load galaxy file, skipping...
Failed to load galaxy file, skipping...


I want gas mass. What is the gas mass? Neutral hydrogen? or any cold gases belong to the host? See Lagos +15

In [37]:
# figure is pickable, but pickle it before clearing the axes. 
pickle.dump(fig, open("fig_interactive.pickle", "wb"))


In [30]:
s_idx_branch + "/" + s_idx_branch + "_" + str(nout)

'28304/28304_30'

determine pericenter.

In [ ]:
# relative distance
for branch in allbranches[1:2]:
    i_peri = get_pericenter(trunk_counter_part(trunk, branch), branch)
    

In [ ]:
# Measure galaxy ellipticity and size?  

In [ ]:
def get_orbit_ang(primary, secondary):
    rel_pos = 


In [ ]:
def get_merging_system():
    
    
    

In [ ]:
def find_merger_epochs(alltrees,
                       idx_all,
                       mpgs,
                       nout_ini=37,
                       dist_gal_scale_in=2.0,
                       dist_gal_scale_out =3.0,
                       min_mass_ratio = 0.05,
                       mass_ratio="early",
                       verbose=False,
                       do_plot = False,
                       pdf_fname='./merger_ratio.pdf',
                       max_rgal=40):
    """
    Parameters
    ----------
    dist_gal_scale 
        if two galaxies are closer than dist_gal_scale * (sum of raidus of the two),
        that epoch is the nout_init_merger.
    nout_ini
        blabla
    """
    gal_list=[]
    mr_list=[]
    nout_list=[]
    nout_ini_list=[] # initial time when two halos(Galaxy stellar components in this case) overlap. 

    #for idx in idx_all:
    if do_plot:
        from matplotlib.backends.backend_pdf import PdfPages
        fig, ax = plt.subplots(1, sharex=True)
        pdf = PdfPages(pdf_fname)
    else:
        ax = None

    for gal in mpgs:
        idx = gal.data['idx'][0]
        # full tree of a galaxy
        atree = ctu.extract_a_tree(alltrees.data, idx)

        # main progenitor tree
        main = ctu.extract_main_tree(atree, idx)
        main_nout = main['nout'].flatten()
        i_nout_ok = main_nout > nout_ini
        main = main[i_nout_ok]
        #x_nout = x_nout[i_nout_ok]
        pos = np.zeros((3,len(main)))
        pos[0,:] = main['x']
        pos[1,:] = main['y']
        pos[2,:] = main['z']


        ## 왜 len(main)?
        mass_ratios_single = np.zeros(len(main))
        nout_inits = np.zeros(len(main))
        #print("log M* ={}".format(np.log10(gal.data["mstar"][0])))

        ## Substitute main["r"] with gal.data["reff"]
        if len(main) < len(gal.smoothed_r):
            main["r"] = gal.smoothed_r[:len(main)]
        elif len(main) > len(gal.smoothed_r):
            main["r"][:len(gal.smoothed_r)] = gal.smoothed_r
        elif len(main) == len(gal.smoothed_r):
            main["r"] = gal.smoothed_r

        for i, nout in enumerate(main['nout']):
            # merger ratio
            # First, check if there are multiple progenitors.
            i_prgs = np.where(atree['desc_id'] == main['id'][i])[0]

            # multiple prgs = merger
            if len(i_prgs) > 1:
                if verbose:
                    print("idx:{}, {} Progenitors at nout = {}".format(main["id"][0], len(i_prgs), nout))

                #print("i, inout_mpgs", i, np.where(gal.nouts == nout)[0])
                #print(gal.data["reff"][i])

                # Mass ratio must be calculated inside get_merger_info. 

                id_prgs = atree['id'][i_prgs]
                mass_prgs = atree['m'][i_prgs]
                #m_r = mass_prgs / max(mass_prgs)

                # Progenitor with maximum mass at the FINAL COALESCENCE is the main progenitor.
                # Others are satellites.
                sats = id_prgs[mass_prgs < max(mass_prgs)]
                
                mass_ratios_now=[]
                nout_inits_now=[]

                # loop over satellites at a given nout.
                for this_sat in sats:
                    n_i_t, mass_this_sat = get_merger_info(main, atree, this_sat,
                                                           dist_gal_scale_in=dist_gal_scale_in,
                                                           dist_gal_scale_out = dist_gal_scale_out,
                                                           do_plot=do_plot,
                                                           ax=ax,
                                                           max_rgal=max_rgal)

                    mass_ratio = mass_this_sat / max(mass_prgs)
                    if do_plot:
                        ax.text(40, 600, "{:.3f}".format(mass_ratio))
                        pdf.savefig()
                        ax.clear()
                    if mass_ratio > min_mass_ratio:
                        nout_inits_now.append(n_i_t)
                        mass_ratios_now.append(mass_ratio)
                if len(nout_inits_now) > 0:
                    i_main_merger = np.argmax(np.array(mass_ratios_now))
                    mass_ratios_single[i] = mass_ratios_now[i_main_merger]
                    nout_inits[i] = nout_inits_now[i_main_merger]
                else:
                    mass_ratios_single[i] = -1
                    nout_inits[i] = -1

                #if verbose:
                #    print(" Mass ratios {} at nout = {}: ".format(m_r, nout_inits[i]))
                #if do_plot:
                #    pdf.savefig()
                    #ax.clear()

            else:
                mass_ratios_single[i] = 0
            ##--------------------------------------------------

        ind_ok = np.where(mass_ratios_single > 0.01)[0]
        if len(ind_ok) > 0:
            # if a satellite oscillates around the host, 
            # it could be identified as multiple mergers with short time interval. 
            # leave only the first passage / merger.
            # No, it doesn't happen in ConsistentTrees.

            #good =[]
            #for i in range(len(ind_ok)-1):
            #    if ind_ok[i+1] > ind_ok[i] + 2:
            #        good.append(ind_ok[i])
            #good.append(ind_ok[-1])
            #ind_ok = good
            mr = 1./mass_ratios_single[ind_ok]

            gal_list.append(idx)
            mr_list.append(mr)
            nout_list.append(main_nout[ind_ok])
            nout_ini_list.append(nout_inits[ind_ok])
            print(idx)
    if do_plot:
        pdf.close()

    inds=[]
    for i, gal in enumerate(mpgs):
        galid = gal.data['idx'][0]
        ind = np.where(galid == gal_list)[0]
        if len(ind) > 0:
            inds.append(i)
            merger = Merger()
            merger.mr = mr_list[ind]
            merger.nout = nout_list[ind]
            merger.nout_ini = nout_ini_list[ind]
            gal.merger = merger
        else:
            gal.merger = None



In [ ]:
def body(clusters,
         dist_gal_scale_in=5,
         dist_gal_scale_out=10,
         dt_before=0.5,
         dt_after=0.5,
         dt_settle=0.5,
         load=False,
         nout_ini = 37,
         filter_small=True,
         min_mass_ratio = 0.05,
         measure_delta_savefig=False,
         find_merger_epoch_plot=False,
         cdir=""):

    suffix = "_{}_{}_{}_{}_{}_{}_{}".format(dist_gal_scale_in,
    dist_gal_scale_out,dt_before,dt_after,dt_settle,nout_ini,
                                           min_mass_ratio)
    if filter_small: 
        suffix = suffix + "_filtered_"

    if load:
        return pickle.load(open("main_prgs_final_augmented" + suffix + ".pickle", 'rb'))
        
    else:
        mpgs = []
        for cluster in clusters:
            print(cluster)
            wdir = base + cluster + '/'

            # Serialize catalogs. -> Only main galaxies
            # main galaxy list
            alltrees = ctu.load_tree(wdir, is_gal=True)
            ad = alltrees.data
            tn = ad[ad['nout'] == nout_fi]

            cat = load_cat(wdir + cdir + 'catalog' + str(nout_fi) + '.pickle')
            #idx_all = [tn['id'][tn['Orig_halo_id'] == id_final][0] for id_final in cat['id']]
            idx_all = cat['idx'][cat["idx"] > 0].astype(int) # why idx are float???

            mpg_tmp = []
            for i, idx in enumerate(idx_all):
                #print(i, idx)

                mpg_tmp.append(MainPrg(ad, idx))
            #    mpg_tmp =[MainPrg(ad, idx) for idx in idx_all]
            for nout in range(nout_ini, nout_fi + 1):
                cat = load_cat(wdir + cdir + 'catalog' + str(nout) + '.pickle')
                for gal in mpg_tmp:
                    gal.set_data(cat, nout)
                    gal.cluster = int(cluster)
        #        print(nout)
            # get rid of galaxies with too short tree.
            mpg_tmp = [gg for gg in mpg_tmp if sum(gg.data["reff"] > 0) > minimum_good_snap]
            for gal in mpg_tmp:
                gal.fill_missing_data()
                gal.clip_non_detection()
                gal.smoothed_lambda_org = mma.smooth(gal.data["lambda_r"], window_len=15)[:-1]
                gal.smoothed_r = mma.smooth(gal.data["reff"], window_len=15)[:-1]
                gal.smoothed_lambda = mma.smooth(l_at_smoothed_r(gal, npix_per_reff=5), window_len=15)[:-1]

            # save for each cluser
            with open(wdir + "main_prgs" + suffix + ".pickle", "wb") as f:
                pickle.dump(mpg_tmp, f)    
                
            # Find_merger_epochs needs smoothed_r
            find_merger_epochs(alltrees,
                               idx_all,
                               mpg_tmp,
                               nout_ini=nout_ini,
                               dist_gal_scale_in=dist_gal_scale_in,
                               dist_gal_scale_out=dist_gal_scale_out,
                               min_mass_ratio = min_mass_ratio,
                               mass_ratio='early',
                               verbose=False,
                               do_plot=find_merger_epoch_plot,
                               max_rgal=40,
                               pdf_fname=str(cluster) + "merger_ratio_epoch" + suffix + ".pdf")
